# Generate Autopology based inputs and data for a condensed phase system 

Currently, the functionality of incoporating bond priors is very limited you can compute bond, angle, and dihedral terms, but you need to make sure that you are using unwrapped cooridinates 

In [1]:
import sys
sys.path.append("../..")
sys.path.append("..")
import sys
import numpy as np
from importlib import reload
import networkx as nx
import torch
from torch.utils.data import DataLoader


In [17]:
from tutorials.data.htvs_snippet import get_mol_ref
import nff.data as d
from nff.data import Dataset, split_train_validation_test, collate_dicts, sparsify_tensor
import pickle

In [3]:
# load ethane data
props = pickle.load( open( "./data/ethane_data.pkl", "rb" ) )
props['offsets'] = [sparsify_tensor(offset.matmul(torch.Tensor(props["cell"][i]))) for i, offset in enumerate(props['offsets'])]
props['smiles'] = ['CC'] * len(props['nxyz'])
dataset = d.Dataset(props.copy(), units='kcal/mol')


In [4]:
# construct bond list 
bond_dic = {'CC': [[2 * i, 2 * i + 1] for i in range(64)]}

In [5]:
# generate topologies
dataset.generate_topologies(bond_dic)

In [6]:

# number of features in the autopology feature vector
n_autopology_features = 256

autopology_params = {
    "n_features": n_autopology_features,
    "n_convolutions": 4,
    "conv_type": "double_node", # this type of convolution is a concatenation of features from
                                # both the node and the nodes it's bonded to. Another option is
                                # single_node, which doesn't use a concatenation
    "conv_update_layers": [{'name': 'linear', 'param' : {'in_features': int(2*n_autopology_features),
                                                    'out_features': n_autopology_features}},
                      {'name': 'Tanh', 'param': {}},
                      {'name': 'linear', 'param' : {'in_features': n_autopology_features,
                                              'out_features': n_autopology_features}},
                      {'name': 'Tanh', 'param': {}}


            ],
    "readout_hidden_nodes": [40, 20], # the number of nodes in each of the hidden layers 
                                      # of the readout network. The readout network acts on
                                      # the features after they have been convolved.
    # types of classical priors to use
    "bond_terms": ["morse"],
    # keys we're outputting 
    "output_keys": ["energy"],
    # whether we want the AuTopology weights to be learned
    "trainable_prior": True
}

In [7]:
from nff.train import get_model
CFF = get_model(autopology_params, model_type='AuTopology')

In [8]:
train_loader = DataLoader(dataset, batch_size=2, collate_fn=collate_dicts)

In [9]:
from nff.utils import batch_to
batch = batch_to( next(iter(train_loader)), "cpu")

In [10]:
# compute energy gradient 
CFF(batch)

{'energy': tensor([[2.3039e-06],
         [2.3000e-06]], grad_fn=<AddBackward0>),
 'energy_grad': tensor([[-2.7237e-08,  8.1618e-09,  3.0758e-08],
         [ 2.7237e-08, -8.1618e-09, -3.0758e-08],
         [-1.2082e-08,  1.3074e-09,  4.1434e-08],
         [ 1.2082e-08, -1.3074e-09, -4.1434e-08],
         [ 4.3317e-08,  7.1728e-09, -6.9362e-10],
         [-4.3317e-08, -7.1728e-09,  6.9362e-10],
         [-2.6932e-08,  2.6996e-08, -1.9648e-08],
         [ 2.6932e-08, -2.6996e-08,  1.9648e-08],
         [ 5.4199e-09, -4.0564e-08, -1.5877e-08],
         [-5.4199e-09,  4.0564e-08,  1.5877e-08],
         [-1.9799e-08, -3.5390e-08, -1.1106e-08],
         [ 1.9799e-08,  3.5390e-08,  1.1106e-08],
         [ 5.8733e-09, -2.2782e-08, -3.6634e-08],
         [-5.8733e-09,  2.2782e-08,  3.6634e-08],
         [ 2.8277e-09,  1.1965e-08, -4.0060e-08],
         [-2.8277e-09, -1.1965e-08,  4.0060e-08],
         [-2.4508e-09, -2.0186e-08, -3.7117e-08],
         [ 2.4508e-09,  2.0186e-08,  3.7117e-08],
   

# Using Glue to combine two models 

In [12]:
# combine two models 
params = {
    'n_atom_basis': 256,
    'n_filters': 256,
    'n_gaussians': 32,
    'n_convolutions': 3,
    'cutoff': 5.0,
    'trainable_gauss': True
}

gcn = get_model(params)

In [13]:
from nff.nn.glue import Stack

In [14]:
model_dict = dict()
model_dict['cff'] = CFF
model_dict['gcn'] = gcn

In [15]:
stack = Stack(model_dict)

In [16]:
stack(batch)

{'energy': tensor([[-5.4545],
         [-6.3955]], grad_fn=<AddBackward0>),
 'energy_grad': tensor([[ 1.7611e-01,  1.0464e-01,  8.3522e-02],
         [-6.5141e-02, -2.1364e-01, -3.5581e-02],
         [ 3.4981e-02, -3.1981e-02,  5.0644e-02],
         [ 1.1369e-01,  1.4931e-01, -1.1140e-01],
         [-2.8486e-01, -2.1774e-02,  4.9714e-02],
         [ 1.1916e-02,  2.8975e-02,  7.1015e-02],
         [ 6.9840e-02, -3.5860e-04, -7.3513e-02],
         [ 1.6584e-01, -1.1285e-01, -3.8586e-02],
         [-2.8901e-03, -2.5354e-02,  1.4684e-02],
         [-9.8539e-02,  1.7521e-01,  9.6410e-02],
         [ 2.5327e-01, -1.2697e-01,  1.3183e-01],
         [ 7.4909e-02, -1.2324e-01,  4.5724e-02],
         [-2.0636e-01, -1.1701e-01, -2.4323e-01],
         [ 2.4988e-02, -4.9816e-02, -1.4972e-01],
         [ 4.1699e-02, -8.1151e-02,  1.0207e-01],
         [-6.7674e-03, -4.6665e-04, -6.5736e-04],
         [ 8.9453e-02,  2.3110e-01, -7.6381e-02],
         [-5.8084e-03, -1.3382e-01, -4.8870e-02],
         